# Basic Usage

This tutorial covers the basics of GETTSIM's interface to get you started with the package. GETTSIM enables an almost complete simulation of the German taxes and transfers system which makes it interesting for both students and researchers. Its extensive coverage of taxes and social policies in Germany makes it a valuable educational tool to learn about the current and past German policy environment. Simultaneously, GETTSIM's capabilities to process household data and compute according taxes and transfers make it a powerful tool that can be used for advanced microsimulations.

The interface consists of two central functions:

1. `set_up_policy_environment` which loads a policy environment for a specified date.

2. `compute_taxes_and_transfers` which allows you to compute taxes and transfers given a specified policy environment for household or individual observations.

The following sections give a brief introduction to these two functions using a minimal working example. The necessary packages and GETTSIM functions can be imported into your notebook as follows.

In [ ]:
import json

from gettsim import (
    PolicyEnvironment,
    compute_taxes_and_transfers,
    create_synthetic_data,
)

## Loading Policies with `PolicyEnvironment.for_date`

The function `PolicyEnvironment.for_date` allows you to load the policy environment in Germany for a given date. The function returns an object that composes

- `params` which is a dictionary containing date-specific parameters for the policy environment.

- `functions` which is a dictionary containing functions that are necessary to compute quantities in the taxes and transfers system on the provided date and data.

Below, we load the policy environment for the year 2020. The exact date for this input will be January 1st, 2020. An exact date would be accepted as an input, too.

In [ ]:
environment = PolicyEnvironment.for_date(2020)

The two objects can be passed on to `compute_taxes_and_transfers` with a number of further inputs to compute outputs for a set of data. Both objects are Python dictionaries that hold information required to set up the  policy environment for the specified date.


**Policy Parameters**

`policy_params` is a nested dictionary of parameters grouped by different policy types they capture. The output below shows the keys of the main dictionary. The names indicate the policy group.

In [ ]:
print(*environment.params.keys(), sep="\n")

These keys can be used to extract the exact parametrizations for a given policy group. The example below for instance shows the parameters that concern social insurance saved under the key `sozialv_beitr`.

In [ ]:
params_sozialv_beitr = environment.params["sozialv_beitr"]

# Print parameters in a nice way
print(json.dumps(params_sozialv_beitr, indent=4, default=str, ensure_ascii=False))

**Policy Functions**

The dictionary `policy_functions` contains functions of policy reforms that correspond to the chosen date. The dictionary keys correspond to the variables they help compute for input data.

In [ ]:
print(*environment.params.keys(), sep="\n")

Both parameters and policy functions are mutable, meaning that GETTSIM not only provides the actual policy environments in Germany for a large range of dates, but also supports changing policies. An extended tutorial on parameters can be found [here](parameters.ipynb) and a tutorial on policy functions is provided [here](policy_functions.ipynb).

### Specifying the Date

Dates can be specified in various ways. The function `PolicyEnvironment.for_date` accepts objects of type str, int, and [datetime](https://docs.python.org/3/library/datetime.html) as inputs to specify a date. If only a year is specified, the policy date will be set to the first day of the year i.e. the inputs `"2020"` and `2020` will both return the policy environment for January 1st, 2020. The input `"2020/03"` on the other hand will set up the policy environment for March 1st, 2020 since a month and year are specified. Lastly, it is also possible to use a specific day such as `"2020/03/21"`, which will return the policy environment for March 21st, 2020.

## Computing Outputs with `compute_taxes_and_transfers`

The policy environment specified by `policy_params` and `policy_functions` can then be
applied to simulated or empirical data to compute taxes and transfers for individuals,
households and other groups defined in the German Tax and Transfer law. This is done via
the function `compute_taxes_and_transfers`. The function requires input data and some
further arguments to be specified.

### Data Requirements, Input Columns, and Targets

The data has to fulfill certain requirements in order for GETTSIM to be able to process it properly. Specifically, GETTSIM requires data to be specified as a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) with  columns marking different input variables. GETTSIM parses the column names, which means that the data columns must be named in a specific way for GETTSIM to recognize them as input variables.

There are more than 50 input names that GETTSIM recognizes, as especially transfers in the German system depend on many different variables. There is a detailed list of them [here](https://gettsim.readthedocs.io/en/latest/gettsim_objects/input_variables.html). The information specified in these inputs can be used to compute taxes and transfers for the selected household, or individual data. The required inputs depend on the desired outputs i.e. the data set does not necessarily have to contain all 45 input variables. A small example is illustrated below. 

#### Exemplary Data Set

For exemplary purposes, we now create a data set which consists of a household of two parents and one child using the `create_synthetic_data` helper function.

In [ ]:
data = create_synthetic_data(
    n_adults=2,
    n_children=1,
    specs_constant_over_households={"bruttolohn_m": [2000.0, 1000.0, 0.0]},
)
# Transpose data for better readability
data.T

This minimal example illustrates some of the naming conventions of input columns:

* There are three identifiers: `p_id` identifies a person, and `hh_id` a household. In this case, we data consists of one household.

* `_m` means, that the variable labeled with this suffix is a monthly variable. Variables without this suffix are always on a yearly basis.

* `_hh` means, that the variable is to be interpreted on household level. Other suffixes
  used for grouping are `sn`, `bg`, `eg`, `fg` and `ehe`. Variables without suffixes are always on individual level.

#### Defining Targets

We first have to select targets i.e. output variables that should be computed for our exemplary observation. In this case we select the four types of social insurance contributions the individuals will have to pay based on their specified information.

In [ ]:
# Create list of target variables.
targets = [
    "ges_krankenv_beitr_arbeitnehmer_m",
    "ges_rentenv_beitr_arbeitnehmer_m",
    "arbeitsl_v_beitr_arbeitnehmer_m",
    "ges_pflegev_beitr_arbeitnehmer_m",
]

### Applying `compute_taxes_and_transfers` to Calculate Outputs

Given the information specified above, we now use `compute_taxes_and_transfers` to compute the variables of interest given by `targets` for our `data` in the selected policy environment given by `policy_params` and `policy_functions`. The function returns a pandas.DataFrame where the columns contain the target variables.

In [ ]:
result = compute_taxes_and_transfers(
    data=data,
    environment=environment,
    targets=targets,
)
result.round(2)

Lastly, we can join the results with the input data to save everything in a single pandas.DataFrame.

In [ ]:
data.join(result).T